In [64]:
using JLD2
using MLJ

include("../src/reduction.jl")
include("../src/ensembles.jl")

@JLD2.load "variables/train_dataset.jld2" train_inputs train_targets
@JLD2.load "variables/test_dataset.jld2" test_inputs test_targets
@JLD2.load "variables/cross_val_index.jdl2" cross_val_index


1-element Vector{Symbol}:
 :cross_val_index

### Cargamos los modelos

In [51]:
BaggingClassifier = MLJ.@load BaggingClassifier pkg=MLJScikitLearnInterface verbosity=0
AdaBoost = MLJ.@load AdaBoostStumpClassifier pkg=DecisionTree verbosity=0
EvoTreeClassifier = MLJ.@load EvoTreeClassifier pkg=EvoTrees verbosity=0
kNNClassifier = MLJ.@load KNNClassifier pkg=NearestNeighborModels verbosity=0
XGBoostClassifier = MLJ.@load XGBoostClassifier pkg=XGBoost verbosity=0
LGBMClassifier = MLJ.@load LGBMClassifier pkg=LightGBM verbosity=0
# CatBoostClassifier = MLJ.@load CatBoostClassifier pkg=CatBoost verbosity=0 #No funciona debido a paqueteria

LightGBM.MLJInterface.LGBMClassifier

### Entrenamos las diferentes configuraciones

In [54]:
configs = [Dict("modelType" => :AdaB, "n_iter" => 100), Dict("modelType" => :BaggingC, "n_estimators" => 10), Dict("modelType" => :BaggingC, "n_estimators" => 50), Dict("modelType" => :EvoT, "n_estimators" => 50), Dict("modelType" => :EvoT, "n_estimators" => 100)]
Emodels = [:AdaB, :BaggingC, :EvoT]
metrics = Dict(model => [] for model in Emodels);
for config in configs
    push!(metrics[config["modelType"]], (TrainCrossValEnsembles(config["modelType"], config, (train_inputs, train_targets), cross_val_index), config))
end;

┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasu

### Selección de los mejores de cada tipo

In [65]:
bestconfigs = Dict();
for model in Emodels
    bestres = sort(metrics[model], by=x -> x[1][1][1], rev=true)[1]
    bestconfigs[model] = bestres
end;

### Entrenamiento en conjunto de test de las mejores combinaciones

In [70]:
for model in Emodels[1:2]
    config = bestconfigs[model][2]
    println(config, TrainEnsembles(model, config, ((train_inputs, train_targets), (test_inputs, test_targets))))
end;

┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346


Dict{String, Any}("modelType" => :AdaB, "n_iter" => 100)(0.37855787476280833, 0.9349112426035503, 0.6245059288537549, 0.7488151658767772)
Dict{String, Any}("modelType" => :BaggingC, "n_estimators" => 10)(0.36242884250474383, 0.6258064516129033, 0.4409090909090909, 0.5173333333333333)


┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346
┌ Warning: Levels not explicitly ordered. Using the order [1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\npanb\.julia\packages\StatisticalMeasures\RRiuf\src\confusion_matrices.jl:346


In [ ]:
TrainEnsembles(:XGB, Dict(), ((train_inputs, train_targets), (test_inputs, test_targets)))